<a href="https://colab.research.google.com/github/amontoy6/Entrega-Notebooks-Lab-2-y-3/blob/main/lda_gensim_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paso 0: Dependencias

In [ ]:
#
# Ana Maria Montoya Mesa
# Universidad EAFIT 
# 2023-1
#

In [ ]:
#configuración en google colab de spark y pyspark
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#!pip install smart-open
#ref: https://radimrehurek.com/gensim/
!pip install gensim
!pip install nltk
!pip install pyLDAvis
# descargar datos 'metadata.csv' de: https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge
# https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge?select=metadata.csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 KB 24.9 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.1
    Uninstalling joblib-1.1.1:
      Successfully uninstalled joblib-1.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires joblib~=1.1.0, but you have joblib 1.2.0 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import nltk
import re

In [ ]:
# settings en nltk: tokenizador y stopwords
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
stop_words_nltk = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# cargar datos en pandas
df = pd.read_csv('gdrive/MyDrive/Maestria/st1800-231/datasets/metadata50K.csv')
#df = pd.read_csv('gdrive/MyDrive/Maestria/st1800-231/datasets/metadata50K.csv')

<ipython-input-5-53c77d38fe4b>:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('gdrive/MyDrive/Maestria/st1800-231/datasets/metadata50K.csv')


In [ ]:
# analisis descriptivo de datos

In [ ]:
df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [ ]:
df.shape

(49999, 19)

In [ ]:
df.count()

cord_uid            49999
sha                 24700
source_x            49999
title               49967
doi                 49999
pmcid               34636
pubmed_id           29480
license             49999
abstract            33498
publish_time        49999
authors             45185
journal             47835
mag_id                  0
who_covidence_id        0
arxiv_id                0
pdf_json_files      24700
pmc_json_files      20385
url                 49999
s2_id               15363
dtype: int64

In [ ]:
# Inicio del proceso LDA
# filtrar las columnas de interés
df = df[['cord_uid','title','abstract']]
df.head()

,cord_uid,title,abstract
0,ug7v899j,Clinical features of culture-proven Mycoplasma...,OBJECTIVE: This retrospective chart review des...
1,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...,Inflammatory diseases of the respiratory tract...
2,ejv2xln0,Surfactant protein-D and pulmonary host defense,Surfactant protein-D (SP-D) participates in th...
3,2b73a28n,Role of endothelin-1 in lung disease,Endothelin-1 (ET-1) is a 21 amino acid peptide...
4,9785vg6d,Gene expression in epithelial cells in respons...,Respiratory syncytial virus (RSV) and pneumoni...


In [ ]:
# función general para preparación de datos: tokenización, remoción caracteres especiales
# minusculas.... no incluido stemming y lematización.
def textprep(line):
    tokens = nltk.word_tokenize(str(line))
    tokens = [w.lower() for w in tokens if len(w)>1]
    tokens = [re.sub(r'[^A-Za-z0-9]+','',w) for w in tokens]
    tokens = [w for w in tokens if w not in stop_words_nltk] 
    return tokens

### Construir el BoW (diccionario) de términos

In [ ]:
# Creación del BoW - en gensim es Dictionary
from gensim.corpora import Dictionary
dictionary = Dictionary(df.tokens_title)

In [ ]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in df.tokens_title]

In [ ]:
print(corpus[:5])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)], [(13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)], [(19, 1), (20, 1), (21, 1), (22, 1), (23, 1)], [(13, 1), (14, 1), (24, 1), (25, 1)], [(26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1)]]


In [ ]:
print(dictionary)

Dictionary(50955 unique tokens: ['abdulaziz', 'arabia', 'clinical', 'cultureproven', 'features']...)


### Construir matriz de documentos vs términos

In [ ]:
# libreria para paralelizar
import multiprocessing as mp
import time

t0 = time.time()
pool = mp.Pool(mp.cpu_count())
doc_term_matrix = pool.map(dictionary.doc2bow, [sentence for sentence in df.tokens_title])
pool.close()
print(time.time()-t0)

0.8797049522399902


### Construir modelo LDA

In [ ]:
from gensim.models.ldamulticore import LdaMulticore

t0 = time.time()
lda_model = LdaMulticore(doc_term_matrix, num_topics=20, id2word = dictionary, passes=10, workers=10)
print(time.time()-t0)

556.9428813457489


In [ ]:
def assigntopic(doc):
    vector = lda_model[dictionary.doc2bow(doc)] 
    # opción 1: todos los tópicos ordenados de mayor a menor, podria ser topN tambien asi: return vector[:5] n=5
    vector = sorted(vector, key=lambda item: -item[1])
    # opción 2: asignar el tópico mayor a cada documento
    #vector = max(vector,key=lambda item: item[1])
    return vector

In [ ]:
df['topics'] = df.apply(lambda row: assigntopic(row['tokens_title']), axis=1)
df.head()

,cord_uid,title,abstract,tokens_title,topics
0,ug7v899j,Clinical features of culture-proven Mycoplasma...,OBJECTIVE: This retrospective chart review des...,"[clinical, features, cultureproven, mycoplasma...","[(10, 0.5953316), (17, 0.34038267)]"
1,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...,Inflammatory diseases of the respiratory tract...,"[nitric, oxide, proinflammatory, mediator, lun...","[(1, 0.66350687), (16, 0.20792168)]"
2,ejv2xln0,Surfactant protein-D and pulmonary host defense,Surfactant protein-D (SP-D) participates in th...,"[surfactant, proteind, pulmonary, host, defense]","[(13, 0.3476957), (12, 0.32958257), (17, 0.181..."
3,2b73a28n,Role of endothelin-1 in lung disease,Endothelin-1 (ET-1) is a 21 amino acid peptide...,"[role, endothelin1, lung, disease]","[(16, 0.52431625), (14, 0.29568374), (8, 0.010..."
4,9785vg6d,Gene expression in epithelial cells in respons...,Respiratory syncytial virus (RSV) and pneumoni...,"[gene, expression, epithelial, cells, response...","[(9, 0.88125)]"


### Ejemplos de tópicos del modelo

In [ ]:
# Mostrar los términos y sus pesos de un documento
print(list(lda_model[doc_term_matrix[0]]))

# Mostrar los términos más relevantes de los tópicos más relevantes tópico y sus pesos
print(lda_model.print_topics(num_topics=10, num_words=3))

[(10, 0.5954039), (17, 0.3403104)]
[(9, '0.034*"virus" + 0.017*"protein" + 0.014*"human"'), (16, '0.045*"disease" + 0.039*"covid19" + 0.035*"coronavirus"'), (19, '0.078*"review" + 0.045*"systematic" + 0.021*"metaanalysis"'), (17, '0.058*"respiratory" + 0.026*"syndrome" + 0.023*"acute"'), (3, '0.023*"study" + 0.023*"patients" + 0.021*"laparoscopic"'), (8, '0.026*"cancer" + 0.009*"treatment" + 0.009*"imaging"'), (10, '0.034*"" + 0.033*"care" + 0.033*"covid19"'), (15, '0.011*"index" + 0.011*"volume" + 0.008*"middle"'), (4, '0.016*"ablation" + 0.010*"vaccines" + 0.010*"radiofrequency"'), (5, '0.049*"case" + 0.036*"report" + 0.010*"failure"')]


In [ ]:
lda_topic_assignment = [max(p,key=lambda item: item[1]) for p in lda_model[corpus]]

## Visualización de todos los tópicos

In [ ]:
!pip install pyLDAvis


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 20.7 MB/s eta 0:00:00


In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

t0 = time.time()
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, doc_term_matrix, dictionary, sort_topics = False)
print(time.time()-t0)
vis

/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


59.22214412689209


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0     -0.022422  0.006099       1        1  3.979766
1     -0.165212 -0.105509       2        1  7.407098
2      0.110631  0.035080       3        1  4.224194
3     -0.075058  0.250242       4        1  8.019525
4      0.106533  0.002157       5        1  3.904744
5     -0.000546  0.095626       6        1  4.434697
6      0.027872 -0.014973       7        1  4.012087
7     -0.123815 -0.126892       8        1  4.559874
8      0.011697  0.119924       9        1  4.235520
9      0.247353 -0.071651      10        1  8.938631
10    -0.153913 -0.085810      11        1  4.988082
11    -0.039614 -0.005858      12        1  4.154839
12    -0.048940 -0.007351      13        1  5.335628
13     0.169423 -0.024942      14        1  4.974603
14     0.096403 -0.020344      15        1  2.935628
15    -0.031746  0.079519      16        1  4.078114
16    -0.096245  0.005891      17        1  5.903721
17     0.041475 -0.010332      18        1  5.557347
18     0.020163 -0.154833      19        1  3.777392
19    -0.074041  0.033956      20        1  4.578510, topic_info=              Term         Freq        Total Category  logprob  loglift
329         review  1525.000000  1525.000000  Default  30.0000  30.0000
10783      covid19  6061.000000  6061.000000  Default  29.0000  29.0000
505    respiratory  1502.000000  1502.000000  Default  28.0000  28.0000
86           virus  2286.000000  2286.000000  Default  27.0000  27.0000
405     systematic   856.000000   856.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
308          among    51.196054   514.981029  Topic20  -5.9008   0.7753
171          using    53.514482  1174.425624  Topic20  -5.8565  -0.0048
4075        stroke    47.931421   380.193259  Topic20  -5.9667   1.0129
121       patients    51.120666  2455.509475  Topic20  -5.9023  -0.7881
331            use    48.599281   730.091551  Topic20  -5.9528   0.3742

[1504 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
239        3  0.004214          
239        4  0.001405          
239       10  0.001405          
239       11  0.984675          
239       15  0.001405          
...      ...       ...       ...
3160      14  0.012700      york
3160      16  0.012700      york
19465      3  0.864336   younger
4520       1  0.870427  zoonoses
4520      16  0.048357  zoonoses

[6359 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

### Guardar la visualización en un archivo HTML

In [ ]:
pyLDAvis.save_html(vis, 'lda_visualization.html')

In [ ]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamulticore import LdaModel

# Create a corpus from a list of texts
common_dictionary = Dictionary(common_texts)
common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=10)

/usr/local/lib/python3.9/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.9/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.9/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [ ]:
print(len(common_texts))

9


In [ ]:
for t in common_texts:
    print(t)

['human', 'interface', 'computer']
['survey', 'user', 'computer', 'system', 'response', 'time']
['eps', 'user', 'interface', 'system']
['system', 'human', 'system', 'eps']
['user', 'response', 'time']
['trees']
['graph', 'trees']
['graph', 'minors', 'trees']
['graph', 'minors', 'survey']


In [ ]:
for text in common_texts:
    print(common_dictionary.doc2bow(text))

[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]


In [ ]:
print(lda)

LdaModel(num_terms=12, num_topics=10, decay=0.5, chunksize=2000)


In [ ]:
# Create a new corpus, made of previously unseen documents.
other_texts = [
    ['computer', 'time', 'graph'],
    ['survey', 'response', 'eps','trees'],
    ['human', 'system', 'computer']
]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]
unseen_doc = other_corpus[1]
vector = lda[unseen_doc]  # get topic probability distribution for a document

In [ ]:
print(vector)

[(0, 0.4199865), (1, 0.020000009), (2, 0.22000666), (3, 0.020000009), (4, 0.21999094), (5, 0.020005567), (6, 0.020000009), (7, 0.020000009), (8, 0.020000009), (9, 0.02001035)]


In [ ]:
for text in other_texts:
    print(text)

['computer', 'time', 'graph']
['survey', 'response', 'eps', 'trees']
['human', 'system', 'computer']


In [ ]:
for text in other_texts:
    print(common_dictionary.doc2bow(text))
for text in other_texts:
    print()

[(0, 1), (6, 1), (10, 1)]
[(3, 1), (4, 1), (8, 1), (9, 1)]
[(0, 1), (1, 1), (5, 1)]





In [ ]:
def assigntopic(doc):
    vector = lda[doc] 
    return vector

In [ ]:
other_texts = [
    ['computer', 'time', 'graph'],
    ['survey', 'response', 'eps','trees'],
    ['human', 'system', 'computer']
]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]
unseen_doc = other_corpus[1]
vector = lda[unseen_doc]  # get topic probability distribution for a document